In [8]:
import cv2
import os
import mediapipe as mp
import numpy as np

In [9]:
mp_holistic = mp.solutions.holistic # Holistic model
mp_drawing = mp.solutions.drawing_utils # Drawing utilities

def mediapipe_detection(image, holistic_model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # COLOR CONVERSION BGR 2 RGB
    image.flags.writeable = False                  # Image is no longer writeable
    results = holistic_model.process(image)        # Make prediction
    image.flags.writeable = True                   # Image is now writeable 
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # COLOR COVERSION RGB 2 BGR
    return image, results

def extract_keypoints(results):
    lh = np.array([[res.x, res.y] for res in results.left_hand_landmarks.landmark]) if results.left_hand_landmarks else np.zeros((21, 2))
    return lh

def draw_styled_landmarks(image, results):
    # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                             ) 
    # Draw right hand connections  
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                             ) 

In [ ]:
def normalize_keypoints(keypoints):
    # Kiểm tra nếu có đủ số điểm (21 điểm cho mỗi bàn tay)
    if keypoints.shape[0] != 21:
        raise ValueError(f"Số lượng điểm keypoints không hợp lệ: {keypoints.shape[0]}")

    # Cổ tay là điểm đầu tiên trong keypoints (index 0)
    wrist = keypoints[0]
    
    # Dịch các điểm sao cho cổ tay trở thành gốc tọa độ (0, 0)
    normalized_keypoints = []
    for point in keypoints:
        normalized_point = (point[0] - wrist[0], point[1] - wrist[1])  # Chỉ cần dịch x, y
        normalized_keypoints.append(normalized_point)
    
    # Chuyển sang numpy array để dễ dàng tính toán min và max
    normalized_keypoints = np.array(normalized_keypoints)
    
    # Tính toán min và max cho x và y
    x_min, y_min = np.min(normalized_keypoints, axis=0)
    x_max, y_max = np.max(normalized_keypoints, axis=0)
    
    # Tránh chia cho 0 nếu max - min = 0
    if (x_max - x_min) == 0:
        print("Cảnh báo: Tọa độ x không thay đổi, bỏ qua chuẩn hóa x.")
        x_min, x_max = 0, 1  # Cứ để giá trị x giữ nguyên, hoặc chọn giá trị mặc định
    if (y_max - y_min) == 0:
        print("Cảnh báo: Tọa độ y không thay đổi, bỏ qua chuẩn hóa y.")
        y_min, y_max = 0, 1  # Cứ để giá trị y giữ nguyên, hoặc chọn giá trị mặc định
    
    # Chuyển min và max về dạng numpy array để có thể tính toán đúng
    min_vals = np.array([x_min, y_min])
    max_vals = np.array([x_max, y_max])
    
    # Chuẩn hóa về phạm vi [-1, 1]
    normalized_keypoints = 2 * (normalized_keypoints - min_vals) / (max_vals - min_vals) - 1
    
    return normalized_keypoints


In [38]:
def process_images(data_dir, output_dir):
    # Duyệt qua từng class trong thư mục
    classes = os.listdir(data_dir)
    for action in classes:
        class_folder = os.path.join(data_dir, action)
        if not os.path.isdir(class_folder):
            continue

        # Tạo thư mục lưu keypoints
        npy_class_dir = os.path.join(output_dir, action)
        os.makedirs(npy_class_dir, exist_ok=True)
        print(f"Đang xử lý lớp: {action}")

        # Duyệt qua từng ảnh trong class
        images = [f for f in os.listdir(class_folder) if f.endswith('.jpg')]
        images.sort()  # Đảm bảo xử lý theo thứ tự frame1, frame2, ...
        
        with mp_holistic.Holistic(min_detection_confidence=0.7, min_tracking_confidence=0.7) as holistic:
            for img_idx, image_file in enumerate(images):
                image_path = os.path.join(class_folder, image_file)
                
                # Đọc ảnh
                frame = cv2.imread(image_path)
                if frame is None:
                    print(f"Không thể đọc ảnh: {image_path}")
                    continue

                # Trích xuất đặc trưng
                image, results = mediapipe_detection(frame, holistic)
                keypoints = extract_keypoints(results)
                
                keypoints = normalize_keypoints(keypoints)

                # Lưu đặc trưng vào file .npy
                npy_path = os.path.join(npy_class_dir, f"{img_idx}.npy")
                np.save(npy_path, keypoints)

                print(f"Lớp {action}: Lưu keypoints từ ảnh {image_file} vào {npy_path}")
                
                if cv2.waitKey(10) & 0xFF == ord('q'):
                    cv2.destroyAllWindows()
                    return

    cv2.destroyAllWindows()

In [39]:
# Cấu trúc thư mục
data_dir = 'data'  # Thư mục chứa các lớp (class1, class2, ...)
output_dir = 'data_normalized'  # Thư mục để lưu keypoints dưới dạng .npy
process_images(data_dir, output_dir)

Đang xử lý lớp: movement
Lớp movement: Lưu keypoints từ ảnh frame_000.jpg vào data_normalized\movement\0.npy
Lớp movement: Lưu keypoints từ ảnh frame_001.jpg vào data_normalized\movement\1.npy
Lớp movement: Lưu keypoints từ ảnh frame_002.jpg vào data_normalized\movement\2.npy
Lớp movement: Lưu keypoints từ ảnh frame_003.jpg vào data_normalized\movement\3.npy
Lớp movement: Lưu keypoints từ ảnh frame_004.jpg vào data_normalized\movement\4.npy
Lớp movement: Lưu keypoints từ ảnh frame_005.jpg vào data_normalized\movement\5.npy
Lớp movement: Lưu keypoints từ ảnh frame_006.jpg vào data_normalized\movement\6.npy
Lớp movement: Lưu keypoints từ ảnh frame_007.jpg vào data_normalized\movement\7.npy
Lớp movement: Lưu keypoints từ ảnh frame_008.jpg vào data_normalized\movement\8.npy
Lớp movement: Lưu keypoints từ ảnh frame_009.jpg vào data_normalized\movement\9.npy
Lớp movement: Lưu keypoints từ ảnh frame_010.jpg vào data_normalized\movement\10.npy
Lớp movement: Lưu keypoints từ ảnh frame_011.jpg v